# Homework and bake-off: pragmatic color descriptions

In [1]:
__author__ = "Christopher Potts"
__version__ = "CS224u, Stanford, Summer 2022"

## Contents

1. [Overview](#Overview)
1. [Set-up](#Set-up)
1. [All two-word examples as a dev corpus](#All-two-word-examples-as-a-dev-corpus)
1. [Dev dataset](#Dev-dataset)
1. [Random train–test split for development](#Random-train–test-split-for-development)
1. [Question 1: Improve the tokenizer [1 point]](#Question-1:-Improve-the-tokenizer-[1-point])
1. [Use the tokenizer](#Use-the-tokenizer)
1. [Question 2: Improve the color representations [1 point]](#Question-2:-Improve-the-color-representations-[1-point])
1. [Use the color representer](#Use-the-color-representer)
1. [Initial model](#Initial-model)
1. [Question 3: GloVe embeddings [1 point]](#Question-3:-GloVe-embeddings-[1-point])
1. [Try the GloVe representations](#Try-the-GloVe-representations)
1. [Question 4: Color context [3 points]](#Question-4:-Color-context-[3-points])
1. [Your original system [3 points]](#Your-original-system-[3-points])
1. [Bakeoff [1 point]](#Bakeoff-[1-point])
1. [Submission Instruction](#Submission-Instruction)

## Overview

This homework and associated bake-off are oriented toward building an effective system for generating color descriptions that are pragmatic in the sense that they would help a reader/listener figure out which color was being referred to in a shared context consisting of a target color (whose identity is known only to the describer/speaker) and a set of distractors.

The notebook [colors_overview.ipynb](colors_overview.ipynb) should be studied before work on this homework begins. That notebook provides backgroud on the task, the dataset, and the modeling code that you will be using and adapting.

The homework questions are more open-ended than previous ones have been. Rather than asking you to implement pre-defined functionality, they ask you to try to improve baseline components of the full system in ways that you find to be effective. As usual, this culminates in a prompt asking you to develop a novel system for entry into the bake-off. In this case, though, the work you do for the homework will likely be directly incorporated into that system (not required, but an efficient way to work at the very least).

## Set-up

See [colors_overview.ipynb](colors_overview.ipynb) for set-up in instructions and other background details.

In [2]:
from colors import ColorsCorpusReader
from nltk.translate.bleu_score import corpus_bleu
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from torch_color_describer import ContextualColorDescriber
from torch_color_describer import create_example_dataset

import utils
from utils import START_SYMBOL, END_SYMBOL, UNK_SYMBOL

In [3]:
utils.fix_random_seeds()

In [4]:
COLORS_SRC_FILENAME = os.path.join(
    "data", "colors", "filteredCorpus.csv")

## All two-word examples as a dev corpus

So that you don't have to sit through excessively long training runs during development, I suggest working with the two-word-only subset of the corpus until you enter into the late stages of system testing.

In [215]:
dev_corpus = ColorsCorpusReader(
    COLORS_SRC_FILENAME,
#     word_count=2,
    normalize_colors=True)

In [216]:
dev_examples = list(dev_corpus.read())

This subset has about one-third the examples of the full corpus:

In [217]:
len(dev_examples)

46994

We __should__ worry that it's not a fully representative sample. Most of the descriptions in the full corpus are shorter, and a large proportion are longer. So this dataset is mainly for debugging, development, and general hill-climbing. All findings should be validated on the full dataset at some point.

## Dev dataset

The first step is to extract the raw color and raw texts from the corpus:

In [218]:
dev_rawcols, dev_texts = zip(*[[ex.colors, ex.contents] for ex in dev_examples])

The raw color representations are suitable inputs to a model, but the texts are just strings, so they can't really be processed as-is. Question 1 asks you to do some tokenizing!

## Random train–test split for development

For the sake of development runs, we create a random train–test split:

In [219]:
dev_rawcols_train, dev_rawcols_test, dev_texts_train, dev_texts_test = \
    train_test_split(dev_rawcols, dev_texts)

## Question 1: Improve the tokenizer [1 point]

This is the first required question – the first required modification to the default pipeline.

The function `tokenize_example` simply splits its string on whitespace and adds the required start and end symbols:

In [10]:
# Moo implementation
# import re

# WORD_RE_STR = r"""
# (?:[a-z][a-z'\-_]+[a-z])       # Words with apostrophes or dashes.
# |
# (?:[+\-]?\d+[,/.:-]\d+[+\-]?)  # Numbers, including fractions, decimals.
# |
# (?:[\w_]+)                     # Words without apostrophes or dashes.
# |
# (?:\.(?:\s*\.){1,})            # Ellipsis dots.
# |
# (?:\*{1,})                     # Asterisk runs.
# |
# (?:\S)                         # Everything else that isn't whitespace.
# """

# WORD_RE = re.compile(r"(%s)" % WORD_RE_STR, re.VERBOSE | re.I | re.UNICODE)

# def segmenter(word):
#     for ending in ['er', 'est', 'ish']:
#         if word.endswith(ending):
#             return [word[:-len(ending)], '+' + ending]
#     return [word]

# def tokenize_example(s):
#     #s = s.lower()
#     words = WORD_RE.findall(s)
#     words = [w.lower() for w in words]
#     heuristic_ending_toknize = [seg for w in words for seg in segmenter(w)]
#     return [START_SYMBOL] + heuristic_ending_toknize + [END_SYMBOL]

#     #return [START_SYMBOL] + s.split() + [END_SYMBOL]

In [265]:
# Doggo implementation
# replacements = [
#     # things to remove
#     (".", ""),
#     ("'", ""),
#     ("!", ""),
#     ("(", ""),
#     (")", ""),
#     ("?", ""),
#     (",", ""),
#     # things to replace
#     ("/", " "),
#     ("-", " "),
# ]

# weird_answers = [
#     "oh boy",
#     "no worries",
#     "thank you",
#     "you too",
#     "no kidding",
#     "wow ummmm",
#     "good job",
#     "nice job",
#     "great job",
#     "tough one",
# ]

# weird_tokens = set([
#     "hi",
#     ":",
#     "ok",
#     "oh",
#     "sorry",
#     "lol",
#     "no",
#     "nice",
#     "d",
#     "yes",
#     "for",
#     "wat",
#     "i",
#     "thanks",
#     "um",
#     "yeah",
# ])

# misspellings = [
#     ("brigher", "brighter"),
#     ("puirple", "purple"),
#     ("grn", "green"),
#     ("brigh", "bright"),
#     ("orang", "orange"),
#     ("gren", "green"),
#     ("pruple", "purple"),
#     ("gray", "grey"),
#     ("yelllow"), ("yellow")
# ]

# endings_to_remove = [
#     "er",
#     "est",
#     "ish",
# ]

def segmenter(word):
    for ending in ['er', 'est', 'ish']:
        if word.endswith(ending):
            return [word[:-len(ending)], '+' + ending]
    return [word]


def get_tokens(s):
    replacements = [
        # things to remove
        (".", ""),
        ("'", ""),
        ("!", ""),
        ("(", ""),
        (")", ""),
        ("?", ""),
        (",", ""),
        # things to replace
        ("/", " "),
        ("-", " "),
    ]

    weird_answers = [
        "oh boy",
        "no worries",
        "thank you",
        "you too",
        "no kidding",
        "wow ummmm",
        "good job",
        "nice job",
        "great job",
        "tough one",
    ]

    weird_tokens = set([
        "hi",
        ":",
        "ok",
        "oh",
        "sorry",
        "lol",
        "no",
        "nice",
        "d",
        "yes",
        "for",
        "wat",
        "i",
        "thanks",
        "um",
        "yeah",
    ])

    misspellings = [
        ("brigher", "brighter"),
        ("puirple", "purple"),
        ("grn", "green"),
        ("brigh", "bright"),
        ("orang", "orange"),
        ("gren", "green"),
        ("pruple", "purple"),
        ("gray", "grey"),
        ("yelllow"), ("yellow")
    ]

    s = s.lower()
    
    for a, b in replacements:
        s = s.replace(a, b)
    
    for a in weird_answers:
        s = s.replace(a, "")
    
    if s == "" or s == " ":
        return [UNK_SYMBOL]

    words = s.split()

    words = [
        w
        for w in words
        if w not in weird_tokens
    ]
    
    words = [seg for w in words for seg in segmenter(w)]
    words = [w for w in words if w != ""]
    
    for i in range(len(words)):
        for m in misspellings:
            if words[i] == m[0]:
                words[i] = m[1]

    return words


from collections import Counter


def get_most_common():
    tokens = []
    for dev_text in dev_texts:
        for token in get_tokens(dev_text):
            tokens.append(token)

    token_counter = Counter(tokens)
    threshold = 20
    most_common = set([w for w in token_counter.keys() if token_counter[w] >= threshold])
    return most_common

most_common = get_most_common()


def tokenize_example(s):
    # NOTE: THIS NEXT LINE IS JUST TO PASS THE AUTOGRADER, which for some reason
    # says global variables are not defined.
    # `most_common` should be a global variable that's in the line above this fn.
    most_common = set(['up','awesome','can','in','normal','pink','intense','last','mid','seafoam','guess','maroon','work','be','have','peach','lavend','ugly','different','like','camo','leaf','barney','it','before','the','got','cyan','id','deep','weird','aqua','flow','say','off','others','dusty','periwinkle','tru','brown','with','see','rose','looking','click','pea','we','from','want','lavendar','pal','colored','pick','image','looks','your','describe','coral','but','some','pale','rust','clos','about','of','hint','b','flat','yellowy','time','p','hue','puple','light','blue','hard','then','blood','sun','really','vivid','purp','that','here','+er','its','problem','wow','there','sup','square','real','mauve','tell','almost','same','if','similar','oth','teal','haha','girl','prince','im','cement','were','rain','day','just','lite','dirt','greyist','royal','a','okay','leaves','beige','only','slight','target','black','one','isnt','choose','very','medium','vibrant','fluorescent','what','tinted','faded','grey','orangey','those','great','pure','olive','and','highlight','brightness','sky','white','dull','2','hello','is','saturated','second','true','soft','violet','an','bright','on','purpley','wat','too','send','go','right','yellow','green','left','them','golden','neon','good','also','e','most','less','purples','washed','well','purpl','doesnt','pinky','neutral','kind','turquoise','greens','vomit','object','rich','these','dark','natural','than','hot','magenta','again','tough','grays','maybe','any','which','bit','foam','concrete','orange','colors','little','between','girls','lilac','you','dont','how','please','me','tone','oops','muted','without','taupe',':d','know','doing','now','baby','purply','think','mustard','primary','mint','hmm','purple','+ish','soup','3','grass','not','my','would','are','hey','pinks','red','###','middle','stormy','gold','ones','shade','close','color','ugh','blu','lipstick','so','ice','rosy','salmon','was','navy','grapes','redd','lime','bold','sand','sure','for','puke','blues','sea','or','yay','yello','bad','look','cent','all','tones','blye','least','did','eith','kinda','caca','been','at','tinge','fun','nor','yep','warm','this','tann','apple','pumpkin','yea','mixed','box','minty','youre','+est','tan','two','dirty','regular','non','side','shades','to','am','slightly','army','as','tint','girly','do','khaki','toned','pastel','brick','turqoise','three','drab','grape','mix','fuschia','they','more','slate','out','ocean','clay','has','sage','sort','thats','plum'])
    tokens = [
        token
        if token in most_common
        else UNK_SYMBOL
        for token in get_tokens(s)
    ]
    return [START_SYMBOL] + tokens + [END_SYMBOL]

In [266]:
tokenize_example(dev_texts_train[376])

['<s>', 'bright', 'blue', '</s>']

__Your task__: Modify `tokenize_example` so that it does something more sophisticated with the input text. 

__Notes__:

* There are useful ideas for this in [Monroe et al. 2017](https://transacl.org/ojs/index.php/tacl/article/view/1142)
* There is no requirement that you do word-level tokenization. Sub-word and multi-word are options.
* This question can interact with the size of your vocabulary (see just below), and in turn with decisions about how to use `UNK_SYMBOL`.

__Important__: don't forget to add the start and end symbols, else the resulting models will definitely be terrible! The following test will check that your tokenizer has this property:

In [235]:
def test_tokenize_example(func):
    s = "A test string"
    result = func(s)
    assert all(isinstance(tok, str) for tok in result), \
        "The tokenizer must return a list of strings."
    assert result[0] == START_SYMBOL, \
        "The tokenizer must add START_SYMBOL as the first token."
    assert result[-1] == END_SYMBOL, \
        "The tokenizer must add END_SYMBOL as the final token."

In [236]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_tokenize_example(tokenize_example)

## Use the tokenizer

Once the tokenizer is working, run the following cell to tokenize your inputs:

In [237]:
dev_seqs_train = [tokenize_example(s) for s in dev_texts_train]

dev_seqs_test = [tokenize_example(s) for s in dev_texts_test]

In [238]:
# Filter out all train seqs with only UNK symbols
keep_idx = set([
    i for i, seq in enumerate(dev_seqs_train)
    if seq.count(UNK_SYMBOL) != len(seq) - 2
])
dev_seqs_train = [
    seq for i, seq in enumerate(dev_seqs_train)
    if i in keep_idx
]

We use only the train set to derive a vocabulary for the model:

In [239]:
dev_vocab = sorted({w for toks in dev_seqs_train for w in toks})

dev_vocab += [UNK_SYMBOL]

It's important that the `UNK_SYMBOL` is included somewhere in this list. In test examples, words not seen in training will be mapped to `UNK_SYMBOL`. 

Conceptual note: If you model's vocab is the same as your train vocab, then `UNK_SYMBOL` will never be encountered during training, so it will be a random vector at test time.

In [240]:
len(dev_vocab)

336

## Question 2: Improve the color representations [1 point]

This is the second required pipeline improvement for the assignment. 

The following functions do nothing at all to the raw input colors we get from the corpus. 

In [257]:
from itertools import product
import math
import numpy as np


def represent_color_context(colors):

    # Improve me!

    return [represent_color(color) for color in colors]


def orig_represent_color(color):
    return color


def hls_to_hsv(h, l, s):
    v = l + s * min(l, 1-l)
    s = 0 if v == 0 else 2*(1 - l/v)
    return [h, s, v]


def represent_color(color):
    h, l, s = color
    h = 360 * h

    h, s, v = hls_to_hsv(h, l, s)
    h = h / 360
    
    all_nums = []
    for j_, k, l in product((0, 1, 2), repeat=3):    
        f_jkl = np.exp(-2 * math.pi * 1j * (j_ * h + k * s + l * v))
        all_nums.append(f_jkl.real)
        all_nums.append(f_jkl.imag)
    return all_nums

In [242]:
represent_color_context(dev_rawcols_train[0])

[[1.0,
  0.0,
  -0.33873792024529137,
  0.9408807689542255,
  -0.7705132427757893,
  -0.6374239897486896,
  -0.9270897516847403,
  0.3748394220477971,
  -0.0386387492842762,
  -0.9992532447051382,
  0.9532665708316103,
  0.3021305097717584,
  0.7189908153577469,
  -0.6950195734158875,
  0.4103810972040285,
  0.9119141160534898,
  -0.9970140941074936,
  0.07721979118732265,
  0.052335956242943966,
  -0.9986295347545738,
  0.9218631515885007,
  0.38751558645210243,
  -0.6768759696826597,
  0.7360970871197353,
  0.3258055889698112,
  0.9454367869910894,
  -0.9999059987537144,
  -0.013711077869252753,
  0.3516065679474363,
  -0.9361478629875865,
  -0.6564380013939883,
  -0.7543799774158022,
  0.9321420565968265,
  -0.3620927868978319,
  0.024934278144434708,
  0.9996890925549883,
  -0.9945218953682733,
  -0.10452846326765373,
  0.43523109937232735,
  -0.9003187714021936,
  0.6996633405133662,
  0.7144726796328027,
  0.9611924457808014,
  -0.2758787454189271,
  -0.06602332391138564,
  0.997

In [243]:
def cos_sim(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

    
c1 = (0.1, 0.5, 0.9)
c2 = (0.3, 0.5, 0.9)

r1 = represent_color(c1)
r2 = represent_color(c2)

r1_orig = orig_represent_color(c1)
r2_orig = orig_represent_color(c2)

print(f"Original: {cos_sim(r1_orig, r2_orig)}")
print(f"New: {cos_sim(r1, r2)}")

Original: 0.9826202025267203
New: 0.16666666666666713


In [244]:
# def represent_color_context(color):
#     return vectorize_all(color)[0]


# TO DO: check space, add convert to rgb_norm or hsv_norm function? 
# def represent_color_context(colors, resolution=3):
    
# #     return [represent_color(color) for color in colors]
    
# #     print(colors)

#     colors = np.array([colors])
#     #print(colors.shape)
#     # assert len(colors.shape) == 3, colors.shape
#     # assert colors.shape[2] == 3, colors.shape

#     ranges = np.array([361, 101, 101])

#     color_0_1 = colors / (ranges - 1.0)
#     #print(color_0_1)

#     xyz = color_0_1[0] / 2.0
#     xyz[:, 0] *= 2.0
    
#     ax, ay, az = [np.arange(0, g) for g, r in zip(range(resolution), ranges)]
#     gx, gy, gz = np.meshgrid(ax, ay, az)

#     arg = (np.multiply.outer(xyz[:, 0], gx) +
#             np.multiply.outer(xyz[:, 1], gy) +
#             np.multiply.outer(xyz[:, 2], gz))
#     #assert 
#     repr_complex = np.exp(-2j * np.pi * (arg % 1.0)).swapaxes(1, 2).reshape((xyz.shape[0], -1))
#     result = np.hstack([repr_complex.real, repr_complex.imag])
#     # print(len(result))
#     return result

In [245]:
# converted_to_hsv = []
# for color in dev_rawcols_train[0]:
#     converted_to_hsv.append(hsl_to_hsv(color))

# # represent_color_context(converted_to_hsv, space="hsv_norm")
# represent_color_context(converted_to_hsv)

__Your task__: Modify `represent_color_context` and/or `represent_color` to represent colors in a new way.
    
__Notes__:

* You are not required to keep `represent_color`. This might be unnatural if you want to perform an operation on each color trio all at once.
* For that matter, if you want to process all of the color contexts in the entire data set all at once, that is fine too, as long as you can also perform the operation at test time with an unknown number of examples being tested.

* The Fourier-transform method of [Monroe et al. 2016](https://www.aclweb.org/anthology/D16-1243/) and [Monroe et al. 2017](https://transacl.org/ojs/index.php/tacl/article/view/1142) is a proven choice for our task. __It is not required that you implement this.__ However, if you decide to, you might find that the overly terse presentation in the paper is an obstacle. They key thing to see is that the notation $\hat{f}_{jkl}$ is meant to specify a full coordinate system. Thus, you might do something like

  ```
from itertools import product
for j, k, l in product((0, 1, 2), repeat=3):    
    f_jkl = ...
```

  and collect these `f_jkl` values in a list of 27 values. Additionally, in Python, [`2j` produces a value with `real` and `imag` attributes](https://docs.python.org/3.7/library/cmath.html). Each element `f_jkl` should have these components. If you concatenate the `real` and `imag` parts of all the `f_jkl`, you will have a 54-dimensional representation, as in the paper. Remember to start with an HSV representation, and with $h$ in $[0, 360]$, $s$ in $[0, 200]$, and $v$ in $[0, 200]$ (or else do the scaling differently). Note that the values in our corpus are in HLS format, [which are easily converted to HSV](https://en.wikipedia.org/wiki/HSL_and_HSV#HSL_to_HSV).
  
* It's natural to ask why this Fourier transform is useful in the current context. This is a challenging question, and I don't have a complete answer, but here is an intuitive observation: if you consider the raw color representations to be embeddings, then you can see very quickly that our standard geometric notions are totally out of line with our intuitions about the colors themselves. For example, here is a plot where we simply vary the hue dimension while keeping the other dimensions constant:

  <img src="fig/colors-hue-hls.png" alt="A series of very different colors with cosine distances from orange ranging from 0 to 0.19" />

  I've printed the cosine distances from the leftmost color above each patch. They all look pretty similar. Now, you might say, well at least the distances are sort of proportional to how different the colors are from the first. However, that argument seems to crumble when we do the same experiment but now varying the saturation dimension:

  <img src="fig/colors-saturation-hls.png" alt="A series of very similar purple-ish colors with cosine distances from gray-purple ranging from 0 to 0.19" />

  These colors are all quite simular intuitively. Notice, though, that the cosine distances are identical to my first plot. Of course! Cosine distances doesn't care about the nature of these dimensions! The underlying color space is a cylinder, not a regular Euclidean 3d space!
  
  The Fourier transformation that we apply is remapping the colors into approximately the cylindrical space that we want. It is at least capturing some the circular/radial relationships that are inherent in the space. Thus, here are plots corresponding to the above, but now where the colors have been transformed for the cosine comparisons. 
  
  First, the hue variation:
  
  <img src="fig/colors-hue-fourier.png" alt="A series of very different colors with cosine distances from orange that are generally large (near 1.0)" />

  And then saturation:
  
  <img src="fig/colors-saturation-fourier.png" alt="A series of very similar purple-ish colors with cosine distances from gray purple that seem aligned with visual color similarity" />
  
  These distances seem much better aligned with intuitions to me, and I think that's quite general. Thus, even if our networks can in principle learn this remapping, it's very helpful to at least start them closer to where we want them to be.
  
  If you want to go one layer deeper, then the [Zhang and Lu 2002](https://www.sciencedirect.com/science/article/pii/S092359650200084X) paper that Monroe et al. 2016 cite is pretty intuitive. It's for the 2d case, but that actually makes the ideas somewhere more accessible, since they can easily plot the original and remapped feature spaces.

The following test seeks to ensure only that the output of your `represent_color_context` will be compatible with the models we are creating:

In [247]:
def test_represent_color_context(func):
    """`func` should be `represent_color_context`"""
    example = [
        [0.786, 0.58, 0.87],
        [0.689, 0.44, 0.92],
        [0.628, 0.32, 0.81]]
    result = func(example)
    assert len(result) == len(example), \
        ("Color context representations need to represent each color "
         "separately. (We assume the final color is the target.)")
    for i, color in enumerate(result):
        assert all(isinstance(x, float) for x in color), \
            ("All color representations should be lists of floats. "
             "Color {} is {}".format(i, color))

In [248]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_represent_color_context(represent_color_context)

## Use the color representer

The following cell just runs your `represent_color_context` on the train and test sets:

In [249]:
dev_cols_train = [represent_color_context(colors) for colors in dev_rawcols_train]

dev_cols_test = [represent_color_context(colors) for colors in dev_rawcols_test]

In [250]:
dev_cols_train = [
    seq for i, seq in enumerate(dev_cols_train)
    if i in keep_idx
]

At this point, our preprocessing steps are complete, and we can fit a first model.

## Initial model

The first model is configured right now to be a small model run for just a few iterations. It should be enough to get traction, but it's unlikely to be a great model. You are free to modify this configuration if you wish; it is here just for demonstration and testing:

In [168]:
dev_mod = ContextualColorDescriber(
    dev_vocab,
    early_stopping=True)

In [169]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    %time _ = dev_mod.fit(dev_cols_train, dev_seqs_train)
else:
    dev_mod.fit(dev_cols_train, dev_seqs_train)

Stopping after epoch 83. Validation score did not improve by tol=1e-05 for more than 10 epochs. Final error is 23.189407348632812

CPU times: user 2min 55s, sys: 3.51 s, total: 2min 59s
Wall time: 2min 35s


The canonical bake-off evaluation function is `evaluate`. Our primary metric is `listener_accuracy`; the BLEU score is included as a check to ensure that your system is speaking English!

In [170]:
evaluation = dev_mod.evaluate(dev_cols_test, dev_seqs_test)

In [171]:
evaluation.keys()

dict_keys(['listener_accuracy', 'corpus_bleu', 'target_index', 'predicted_index', 'predicted_utterance'])

In [172]:
evaluation['listener_accuracy']

0.791534696228045

In [173]:
dev_mod.listener_accuracy(dev_cols_test, dev_seqs_test)

0.791534696228045

In [174]:
evaluation['corpus_bleu']

0.6976881518889887

In [175]:
bleu, predicted_utterances = dev_mod.corpus_bleu(dev_cols_test, dev_seqs_test)

bleu

0.6976881518889887

In [176]:
evaluation['target_index'][: 5]

[2, 2, 2, 2, 2]

In [177]:
evaluation['predicted_index'][: 5]

[2, 0, 2, 0, 2]

In [178]:
evaluation['predicted_utterance'][: 5]

[['<s>', 'bright', 'purple', '</s>'],
 ['<s>', 'yellow', '+ish', 'green', '</s>'],
 ['<s>', 'pink', '+ish', 'purple', '</s>'],
 ['<s>', 'bright', 'pink', '</s>'],
 ['<s>', 'bright', 'green', '</s>']]

We can also see the model's predicted sequences given color context inputs:

In [179]:
dev_mod.predict(dev_cols_test[: 1])

[['<s>', 'bright', 'purple', '</s>']]

In [180]:
dev_seqs_test[: 1]

[['<s>', '$UNK', '$UNK', '###', 'purple', 'pink', '+ish', '</s>']]

## Question 3: GloVe embeddings [1 point]

The above model uses a random initial embedding, as configured by the decoder used by `ContextualColorDescriber`. This homework question asks you to consider using GloVe inputs. 

__Your task__: Complete `create_glove_embedding` so that it creates a GloVe embedding based on your model vocabulary. This isn't meant to be analytically challenging, but rather just to create a basis for you to try out other kinds of rich initialization.

In [181]:
GLOVE_HOME = os.path.join('data', 'glove.6B')

In [182]:
def create_glove_embedding(vocab, glove_base_filename='glove.6B.50d.txt'):
    # Use `utils.glove2dict` to read in the GloVe file:
    glove_path = os.path.join("./data", "glove.6B", glove_base_filename)
    lookup = utils.glove2dict(glove_path)

    # Use `utils.create_pretrained_embedding` to create the embedding.
    # This function will, by default, ensure that START_TOKEN,
    # END_TOKEN, and UNK_TOKEN are included in the embedding.

    embedding, new_vocab = utils.create_pretrained_embedding(
        lookup, vocab, required_tokens=('$UNK', "<s>", "</s>")
    )


    # Be sure to return the embedding you create as well as the
    # vocabulary returned by `utils.create_pretrained_embedding`,
    # which is likely to have been modified from the input `vocab`.

    return embedding, new_vocab

In [183]:
def test_create_glove_embedding(func):
    vocab = ['NLU', 'is', 'the', 'future', '.', '$UNK', '<s>', '</s>']
    glove_embedding, glove_vocab = func(vocab, 'glove.6B.50d.txt')
    assert isinstance(glove_embedding, np.ndarray), \
        "Expected embedding type {}; got {}".format(
        glove_embedding.__class__.__name__, glove_embedding.__class__.__name__)
    assert glove_embedding.shape == (8, 50), \
        "Expected embedding shape (8, 50); got {}".format(glove_embedding.shape)
    assert glove_vocab == vocab, \
        "Expected vocab {}; got {}".format(vocab, glove_vocab)

In [184]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_create_glove_embedding(create_glove_embedding)

## Try the GloVe representations

The extent to which GloVe is useful will depend heavily on how aligned your tokenization scheme is with the GloVe vocabulary. For example, if you did character-level tokenization, then the GloVe embedding space is not well-aligned with your tokenizer and using GloVe should have little no positive effect.

Let's see if GloVe helped for our development data:

In [185]:
dev_glove_embedding, dev_glove_vocab = create_glove_embedding(dev_vocab)

In [186]:
len(dev_vocab)

131

In [187]:
len(dev_glove_vocab)

131

In [188]:
dev_mod_glove = ContextualColorDescriber(
    dev_glove_vocab,
    embedding=dev_glove_embedding,
    early_stopping=True)

In [189]:
_ = dev_mod_glove.fit(dev_cols_train, dev_seqs_train)

Stopping after epoch 73. Validation score did not improve by tol=1e-05 for more than 10 epochs. Final error is 23.332935094833374

In [190]:
dev_mod_glove.listener_accuracy(dev_cols_test, dev_seqs_test)

0.7898070832133602

You probably saw a small boost, assuming your tokeization scheme leads to good overlap with the GloVe vocabulary. The input representations are larger than in our previous model (at least as I configured things), so we would need to do more runs with higher `max_iter` values to see whether this is worthwhile overall.

## Question 4: Color context [3 points]

The final required homework question is the most challenging, but it should set you up to think in much more flexible ways about the underlying model we're using.

The question asks you to modify various model components in `torch_color_describer.py`. The section called [Modifying the core model](colors_overview.ipynb#Modifying-the-core-model) from the core unit notebook provides a number of examples illustrating the basic techniques, so you might review that material if you get stuck here.

__Your task__: Building on ideas from [Monroe et al. 2017](https://transacl.org/ojs/index.php/tacl/article/view/1142), you will redesign the model so that the target color (the final one in the context) is appended to each input token that gets processed by the decoder. The question asks you to subclass the `Decoder` and `EncoderDecoder` from `torch_color_describer.py` so that you can build models that do this.

__Step 1__: Modify the `Decoder` so that the input vector to the model at each timestep is not just a token representation `x` but the concatenation of `x` with the representation of the target color.

__Notes__:

* You might notice at this point that the original `Decoder.forward` method has an optional keyword argument `target_colors` that is passed to `Decoder.get_embeddings`. Because this is already in place, all you have to do is modify the `get_embeddings` method to use this argument.

* The change affects the configuration of `self.rnn`, so you need to subclass the `__init__` method as well, so that its `input_size` argument accomodates the embedding as well as the color representations.

* You can do the relevant operations efficiently in pure PyTorch using `repeat_interleave` and `cat`, but the important thing is to get a working implementation – you can always optimize the code later if the ideas prove useful to you. 

Here's skeleton code for you to flesh out:

In [191]:
from torch_color_describer import Decoder
import torch
import torch.nn as nn


class ColorContextDecoder(Decoder):
    def __init__(self, color_dim, *args, **kwargs):
        self.color_dim = color_dim
        super().__init__(*args, **kwargs)

        # Fix the `self.rnn` attribute:
        self.rnn = nn.GRU(
            input_size=self.embed_dim + color_dim,
            hidden_size=self.hidden_dim,
            batch_first=True)

    def get_embeddings(self, word_seqs, target_colors=None):
        """
        You can assume that `target_colors` is a tensor of shape
        (m, n), where m is the length of the batch (same as
        `word_seqs.shape[0]`) and n is the dimensionality of the
        color representations the model is using. The goal is
        to attached each color vector i to each of the tokens in
        the ith sequence of (the embedded version of) `word_seqs`.

        """
        m, n = word_seqs.shape
        embs = self.embedding(word_seqs)
        colors_interleaved = torch.repeat_interleave(target_colors, n, dim=0)
        colors_interleaved = colors_interleaved.reshape(m, n, self.color_dim)
        output = torch.cat([embs, colors_interleaved], dim=2)
        return output

Step 1 is the most demanding of the steps in terms of tensor wrangling. It's important to have a clear idea of what you are trying to achieve and to unit test `get_embeddings` so that you can check that it has realized your vision. The following test should help with that:

In [192]:
def test_get_embeddings(decoder_class):
    """
    It's assumed that the input to this will be `ColorContextDecoder`.
    You pass in the class, and the function initalizes it with the test
    parameters.
    """
    dec = decoder_class(
        color_dim=3,   # For these, we mainly want *different*
        vocab_size=10, # dimensions so that we reliably get
        embed_dim=4,   # dimensionality errors if something
        hidden_dim=5)  # isn't working.

    # This step just changes the embedding to one with values
    # that are easy to inspect and definitely will not change
    # between runs:
    dec.embedding = nn.Embedding.from_pretrained(
        torch.FloatTensor([
            [10, 11, 12, 13],
            [14, 15, 16, 17],
            [18, 19, 20, 21]]))

    # These are the incoming sequences -- lists of indices
    # into the rows of `dec.embedding`:
    word_seqs = torch.tensor([
        [0,1,2],
        [2,0,1]])

    # Target colors as small floats that will be easy to track:
    target_colors = torch.tensor([
        [0.1, 0.2, 0.3],
        [0.7, 0.8, 0.9]])

    # The desired return value: one list of tensors for each of
    # the two sequences in `word_seqs`. Each index is replaced
    # with its vector from `dec.embedding` and has the
    # corrresponding color from `target_colors` appended to it.
    expected = torch.tensor([
        [[10., 11., 12., 13.,  0.1,  0.2,  0.3],
         [14., 15., 16., 17.,  0.1,  0.2,  0.3],
         [18., 19., 20., 21.,  0.1,  0.2,  0.3]],

        [[18., 19., 20., 21.,  0.7,  0.8,  0.9],
         [10., 11., 12., 13.,  0.7,  0.8,  0.9],
         [14., 15., 16., 17.,  0.7,  0.8,  0.9]]])

    result = dec.get_embeddings(word_seqs, target_colors=target_colors)

    assert expected.shape == result.shape, \
        "Expected shape {}; got shape {}".format(expected.shape, result.shape)

    assert torch.all(expected.eq(result)), \
        ("Your result has the desired shape but the values aren't correct. "
         "Here's what your function creates; compare it with `expected` "
         "from the test:\n{}".format(result))

In [193]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_get_embeddings(ColorContextDecoder)

__Step 2__: Modify the `EncoderDecoder`. For this, you just need to make a small change to the `forward` method: extract the target colors from `color_seqs` and feed them to the decoder.

In [194]:
from torch_color_describer import EncoderDecoder

class ColorizedEncoderDecoder(EncoderDecoder):

    def forward(self,
            color_seqs,
            word_seqs,
            seq_lengths=None,
            hidden=None,
            targets=None):
        if hidden is None:
            hidden = self.encoder(color_seqs)

        # Extract the target colors from `color_seqs` and
        # feed them to the decoder, which already has a
        # `target_colors` keyword.

        target_colors = color_seqs[:, -1, :]
        output, hidden = self.decoder(word_seqs, seq_lengths=seq_lengths, hidden=hidden, target_colors=target_colors)

        # Your decoder will return `output, hidden` pairs; the
        # following will handle the two return situations that
        # the code needs to consider -- training and prediction.
        if self.training:
            return output
        else:
            return output, hidden

__Step 3__: Finally, as in the examples in [Modifying the core model](colors_overview.ipynb#Modifying-the-core-model), you need to modify the `build_graph` method of `ContextualColorDescriber` so that it uses your new `ColorContextDecoder` and `ColorizedEncoderDecoder`. Here's starter code:

In [195]:
from torch_color_describer import Encoder

class ColorizedInputDescriber(ContextualColorDescriber):

    def build_graph(self):
        # We didn't modify the encoder, so this is
        # just copied over from the original:
        encoder = Encoder(
            color_dim=self.color_dim,
            hidden_dim=self.hidden_dim)

        # Use your `ColorContextDecoder`, making sure
        # to pass in all the keyword arguments coming
        # from `ColorizedInputDescriber`:

        decoder = ColorContextDecoder(
            color_dim=self.color_dim,
            vocab_size=self.vocab_size,
            embed_dim=self.embed_dim,
            hidden_dim=self.hidden_dim,
            embedding=self.embedding,
            freeze_embedding=self.freeze_embedding,
        )

        # Return a `ColorizedEncoderDecoder` that uses
        # your encoder and decoder:
        return ColorizedEncoderDecoder(encoder, decoder)

That's it! Since these modifications are pretty intricate, you might want to use [a toy dataset](colors_overview.ipynb#Toy-problems-for-development-work) to debug it:

In [196]:
def test_full_system(describer_class):
    toy_color_seqs, toy_word_seqs, toy_vocab = create_example_dataset(
        group_size=50, vec_dim=2)

    toy_color_seqs_train, toy_color_seqs_test, toy_word_seqs_train, toy_word_seqs_test = \
        train_test_split(toy_color_seqs, toy_word_seqs)

    toy_mod = describer_class(toy_vocab)

    _ = toy_mod.fit(toy_color_seqs_train, toy_word_seqs_train)

    acc = toy_mod.listener_accuracy(toy_color_seqs_test, toy_word_seqs_test)

    return acc

In [197]:
test_full_system(ColorizedInputDescriber)

Finished epoch 1000 of 1000; error is 0.12230164557695389

1.0

If that worked, then you can now try this model on SCC problems!

## Your original system [3 points]

There are many options for your original system, which consists of the full pipeline – all preprocessing and modeling steps. You are free to use any model you like, as long as you subclass `ContextualColorDescriber` in a way that allows its `evaluate` method to behave in the expected way.

So that we can evaluate models in a uniform way for the bake-off, we ask that you modify the function `evaluate_original_system` below so that it accepts a trained instance of your model and does any preprocessing steps required by your model.

If we seek to reproduce your results, we will rerun this entire notebook. Thus, it is fine if your `evaluate_original_system` makes use of functions you wrote or modified above this cell.

In [210]:
def evaluate_original_system(trained_model, color_seqs_test, texts_test):
    """
    Feel free to modify this code to accommodate the needs of
    your system. Just keep in mind that it will get raw corpus
    examples as inputs for the bake-off.

    """
    # `word_seqs_test` is a list of strings, so tokenize each of
    # its elements:
    tok_seqs = [tokenize_example(s) for s in texts_test]

    col_seqs = [represent_color_context(colors)
                for colors in color_seqs_test]


    # Optionally include other preprocessing steps here. Note:
    # DO NOT RETRAIN YOUR MODEL AS PART OF THIS EVALUATION!
    # It's a tempting step, but it's a mistake and will get
    # you disqualified!

    # The following core score calculations are required:
    evaluation = trained_model.evaluate(col_seqs, tok_seqs)

    return evaluation

If `evaluate_original_system` works on test sets you create from the corpus distribution, then it will work for the bake-off, so consider checking that. For example, this would check that `dev_mod` above passes muster:

In [254]:
my_evaluation = evaluate_original_system(dev_mod, dev_rawcols_test, dev_texts_test)

/Users/cwolff/Documents/cs224u/torch_color_describer.py:680: RuntimeWarning: divide by zero encountered in power
  perp = [np.prod(s)**(-1/len(s)) for s in scores]


In [255]:
my_evaluation['listener_accuracy']

0.8339433143246233

In [256]:
my_evaluation['corpus_bleu']

0.487740744889098

In the cell below, please provide a brief technical description of your original system, so that the teaching team can gain an understanding of what it does. This will help us to understand your code and analyze all the submissions to identify patterns and strategies. We also ask that you report the best **listener_accuracy** score your system got during development, just to help us understand how systems performed overall.

<font color='red'>Please review the descriptions in the following comment and follow the instructions.</font>

In [251]:
# PLEASE MAKE SURE TO INCLUDE THE FOLLOWING BETWEEN THE START AND STOP COMMENTS:
#   1) Textual description of your system.
#   2) The code for your original system.
#   3) The score achieved by your system in place of MY_NUMBER.
#        With no other changes to that line.
#        You should report your score as a decimal value <=1.0
# PLEASE MAKE SURE NOT TO DELETE OR EDIT THE START AND STOP COMMENTS

# NOTE: MODULES, CODE AND DATASETS REQUIRED FOR YOUR ORIGINAL SYSTEM
# SHOULD BE ADDED BELOW THE 'IS_GRADESCOPE_ENV' CHECK CONDITION. DOING
# SO ABOVE THE CHECK MAY CAUSE THE AUTOGRADER TO FAIL.

# START COMMENT: Enter your system description in this cell.
# Final system description:
# - Our tokenizer replaces special tokens, removes meaningless phrases, splits
#   the endings "er" "est" and "ish" into special tokens, and converts all
#   words that appear less than 20 times into `UNK_SYMBOL`. We also fix many
#   spelling mistakes that we found in the dev train corpus.
# - After tokenization, we remove the training examples that only contain unknowns.
# - Our color representation uses Fourier features, similar to Monroe et al. 2016.
# = Our encoder and decoder are both 2 layer GRUs with a hidden dim of 50.
# - After hill climbing to this final model, we train on the full dev set, that
#   includes all descriptions, not just the 2 word ones.
#
# Other things we tried:
# - We tried using Glove embeddings but found that they were not helpful.
# - We experimented with different model sizes (1 layer, 3 layers) and LSTMs.
# = We tried different tokenization strategies, e.g. cutoffs for frequent word
#   count.
# - We experimented with different optimizers and learning rates. RMSprop and
#   SGD both performed worse than Adam. We found that a learning rate of 0.001
#   worked well with Adam.
#
# My peak score was: 0.8343

from torch_color_describer import Decoder
from torch_color_describer import Encoder
from torch_color_describer import EncoderDecoder
import torch
import torch.nn as nn


class BakeoffEncoder(Encoder):
    def __init__(self, *args, num_layers=1, **kwargs):
        super().__init__(*args, **kwargs)
        self.num_layers = num_layers
        self.rnn = nn.GRU(
            input_size=self.color_dim,
            hidden_size=self.hidden_dim,
            num_layers=self.num_layers,
            batch_first=True)


class BakeoffDecoder(Decoder):
    def __init__(self, color_dim, *args, num_layers=1, **kwargs):
        super().__init__(*args, **kwargs)
        self.color_dim = color_dim
        self.num_layers = num_layers

        # Fix the `self.rnn` attribute:
        self.rnn = nn.GRU(
            input_size=self.embed_dim + color_dim,
            hidden_size=self.hidden_dim,
            num_layers=self.num_layers,
            batch_first=True)

    def get_embeddings(self, word_seqs, target_colors=None):
        m, n = word_seqs.shape
        embs = self.embedding(word_seqs)
        colors_interleaved = torch.repeat_interleave(target_colors, n, dim=0)
        colors_interleaved = colors_interleaved.reshape(m, n, self.color_dim)
        output = torch.cat([embs, colors_interleaved], dim=2)
        return output


class BakeoffEncoderDecoder(EncoderDecoder):
    def __init__(self, encoder, decoder, pdrop):
        super().__init__(encoder, decoder)
        self.dropout = nn.Dropout(pdrop)

    def forward(self,
            color_seqs,
            word_seqs,
            seq_lengths=None,
            hidden=None,
            targets=None):
        if hidden is None:
            hidden = self.encoder(color_seqs)

        # Extract the target colors from `color_seqs` and
        # feed them to the decoder, which already has a
        # `target_colors` keyword.
        
        hidden = self.dropout(hidden)

        target_colors = color_seqs[:, -1, :]
        output, hidden = self.decoder(word_seqs, seq_lengths=seq_lengths, hidden=hidden, target_colors=target_colors)

        # Your decoder will return `output, hidden` pairs; the
        # following will handle the two return situations that
        # the code needs to consider -- training and prediction.
        if self.training:
            return output
        else:
            return output, hidden


class BakeoffDescriber(ContextualColorDescriber):
    
    def __init__(self, *args, num_layers=1, pdrop=0.0, **kwargs):
        super().__init__(*args, **kwargs)
        self.num_layers = num_layers
        self.pdrop = pdrop

    def build_graph(self):
        # We didn't modify the encoder, so this is
        # just copied over from the original:
        encoder = BakeoffEncoder(
            num_layers=self.num_layers,
            color_dim=self.color_dim,
            hidden_dim=self.hidden_dim)

        # Use your `ColorContextDecoder`, making sure
        # to pass in all the keyword arguments coming
        # from `ColorizedInputDescriber`:

        decoder = BakeoffDecoder(
            num_layers=self.num_layers,
            color_dim=self.color_dim,
            vocab_size=self.vocab_size,
            embed_dim=self.embed_dim,
            hidden_dim=self.hidden_dim,
            embedding=self.embedding,
            freeze_embedding=self.freeze_embedding,
        )

        # Return a `ColorizedEncoderDecoder` that uses
        # your encoder and decoder:
        return BakeoffEncoderDecoder(encoder, decoder, self.pdrop)


if 'IS_GRADESCOPE_ENV' not in os.environ:
#     base_model = ContextualColorDescriber(
#         dev_vocab,
#         early_stopping=True)
#     base_model.fit(dev_cols_train, dev_seqs_train)
#     base_eval = base_model.evaluate(dev_cols_test, dev_seqs_test)
#     print(f"Base model:")
#     print(f"Listener accuracy: {base_eval['listener_accuracy']}")
#     print(f"BLEU: {base_eval['corpus_bleu']}")

    bakeoff_model = BakeoffDescriber(
        dev_vocab,
        num_layers=2,
        pdrop=0.2,
        early_stopping=True,
    )
    bakeoff_model.fit(dev_cols_train, dev_seqs_train)
    bakeoff_eval = bakeoff_model.evaluate(dev_cols_test, dev_seqs_test)
    print(f"Bakeoff model:")
    print(f"Listener accuracy: {bakeoff_eval['listener_accuracy']}")
    print(f"BLEU: {bakeoff_eval['corpus_bleu']}")

# STOP COMMENT: Please do not remove this comment.

/Users/cwolff/Documents/cs224u/torch_color_describer.py:680: RuntimeWarning: divide by zero encountered in power
  perp = [np.prod(s)**(-1/len(s)) for s in scores]
Finished epoch 1 of 1000; error is 176.13652801513672/Users/cwolff/Documents/cs224u/torch_color_describer.py:680: RuntimeWarning: divide by zero encountered in power
  perp = [np.prod(s)**(-1/len(s)) for s in scores]
Finished epoch 2 of 1000; error is 164.8392686843872/Users/cwolff/Documents/cs224u/torch_color_describer.py:680: RuntimeWarning: divide by zero encountered in power
  perp = [np.prod(s)**(-1/len(s)) for s in scores]
Finished epoch 3 of 1000; error is 161.83184480667114/Users/cwolff/Documents/cs224u/torch_color_describer.py:680: RuntimeWarning: divide by zero encountered in power
  perp = [np.prod(s)**(-1/len(s)) for s in scores]
Finished epoch 4 of 1000; error is 159.67885398864746/Users/cwolff/Documents/cs224u/torch_color_describer.py:680: RuntimeWarning: divide by zero encountered in power
  perp = [np.prod(s)

Finished epoch 38 of 1000; error is 101.60731077194214/Users/cwolff/Documents/cs224u/torch_color_describer.py:680: RuntimeWarning: divide by zero encountered in power
  perp = [np.prod(s)**(-1/len(s)) for s in scores]
Finished epoch 39 of 1000; error is 99.94784259796143/Users/cwolff/Documents/cs224u/torch_color_describer.py:680: RuntimeWarning: divide by zero encountered in power
  perp = [np.prod(s)**(-1/len(s)) for s in scores]
Finished epoch 40 of 1000; error is 98.32935881614685/Users/cwolff/Documents/cs224u/torch_color_describer.py:680: RuntimeWarning: divide by zero encountered in power
  perp = [np.prod(s)**(-1/len(s)) for s in scores]
Finished epoch 41 of 1000; error is 97.06661677360535/Users/cwolff/Documents/cs224u/torch_color_describer.py:680: RuntimeWarning: divide by zero encountered in power
  perp = [np.prod(s)**(-1/len(s)) for s in scores]
Finished epoch 42 of 1000; error is 95.26907634735107/Users/cwolff/Documents/cs224u/torch_color_describer.py:680: RuntimeWarning: d

Stopping after epoch 76. Validation score did not improve by tol=1e-05 for more than 10 epochs. Final error is 49.2242968082428/Users/cwolff/Documents/cs224u/torch_color_describer.py:680: RuntimeWarning: divide by zero encountered in power
  perp = [np.prod(s)**(-1/len(s)) for s in scores]


Bakeoff model:
Listener accuracy: 0.8342837688313899
BLEU: 0.487740744889098


## Bakeoff [1 point]

For the bake-off, we will use our original test set. The function you need to run for the submission is the following, which uses your `evaluate_original_system` from above:

In [252]:
def create_bakeoff_submission(
        trained_model,
        output_filename='cs224u-colors-bakeoff-entry.csv'):
    bakeoff_src_filename = os.path.join(
        "data", "colors", "cs224u-colors-test.csv")

    bakeoff_corpus = ColorsCorpusReader(bakeoff_src_filename)

    # This code just extracts the colors and texts from the new corpus:
    bakeoff_rawcols, bakeoff_texts = zip(*[
        [ex.colors, ex.contents] for ex in bakeoff_corpus.read()])

    # Original system function call; `trained_model` is your trained model:
    evaluation = evaluate_original_system(
        trained_model, bakeoff_rawcols, bakeoff_texts)

    evaluation['bakeoff_text'] = bakeoff_texts

    df = pd.DataFrame(evaluation)
    df.to_csv(output_filename)

In [253]:
# This check ensure that the following code only runs on the local environment only.
# The following call will not be run on the autograder environment.
if 'IS_GRADESCOPE_ENV' not in os.environ:
    pass
    create_bakeoff_submission(bakeoff_model)

This creates a file `cs224u-colors-bakeoff-entry.csv` in the current directory. That file should be uploaded as-is. Please do not change its name.

Only one upload per team is permitted, and you should do no tuning of your system based on what you see in the file – you should not study that file in anyway, beyond perhaps checking that it contains what you expected it to contain. The upload function will do some additional checking to ensure that your file is well-formed.

The nature of our evaluation is such that we have to release the full test set with all labels. Thus, we have to trust you not to make any use of the test set during development. Recall:

1. Only one evaluation is permitted.
1. No additional system tuning is permitted once the bake-off has started.

Systems will be ranked primarily by `listener_accuracy`, but we will also consider their `corpus_bleu` scores. However, the BLEU score is just a simple check that your system is speaking some version of English that corresponds in some meaningful way to the gold descriptions, so you should concentrate on `listener_accuracy`.

People who enter will receive the additional homework point, and people whose systems achieve the top score will receive an additional 0.5 points. We will test the top-performing systems ourselves, and only systems for which we can reproduce the reported results will win the extra 0.5 points.

Late entries will be accepted, but they cannot earn the extra 0.5 points.

## Submission Instruction

Review and follow the [Homework and bake-off code: Formatting guide](hw_formatting_guide.ipynb).
Please do not change the file name as described below.

Submit the following files to Gradescope:

- `hw_colors.ipynb` (this notebook)
- `cs224u-colors-bakeoff-entry.csv` (bake-off output)